#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Load data (users.csv, posts.csv)

In [2]:
users = pd.read_csv('C:/Users/victo/Desktop/IronHack/13-lab-pandas-data-cleaning/your-code/users.csv')

In [3]:
posts = pd.read_csv('C:/Users/victo/Desktop/IronHack/13-lab-pandas-data-cleaning/your-code/posts.csv')

#### 3. Rename id column to user_id

In [4]:
users.rename(columns={'id':'user_id'}, inplace=True)

#### 4. Rename id column to post_id and owner_user_id to user_id

In [5]:
posts.rename(columns={'id':'post_id', 'owner_user_id':'user_id'}, inplace=True)

#### 5. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [6]:
new_users = users[['user_id', 'reputation', 'views', 'up_votes', 'down_votes']]

In [7]:
new_posts = posts[['post_id', 'score', 'user_id', 'view_count', 'comment_count']]

#### 6. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [8]:
users_posts = new_users.merge(new_posts)

In [9]:
users_posts

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
0,450456,7746,3598,256,50,34989087,0,19,0
1,450456,7746,3598,256,50,4142174,0,15,0
2,1517244,4172,224,1362,32,37211628,2,18,0
3,1870509,897,105,86,2,30260248,1,11,4
4,101719,4055,332,174,16,34528617,0,18,0
...,...,...,...,...,...,...,...,...,...
1060,4295902,1,4,0,0,31038301,0,10,0
1061,2856201,11,21,0,0,19348144,0,19,2
1062,5882818,11,3,0,0,35199880,0,20,2
1063,6241297,13,2,0,0,36798683,0,12,0


#### 7. How many missing values do you have in your merged dataframe? 

In [10]:
users_posts.isnull().sum()

user_id          0
reputation       0
views            0
up_votes         0
down_votes       0
post_id          0
score            0
view_count       0
comment_count    0
dtype: int64

There are no missing values

#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [12]:
users_posts.describe()

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
count,1.065000e+03,1065.000000,1065.000000,1065.000000,1065.000000,1.065000e+03,1065.000000,1065.000000,1065.000000
mean,2.280169e+06,7251.176526,615.245070,397.688263,49.415023,3.398846e+07,0.123944,15.606573,0.926761
std,1.833959e+06,24021.612286,1497.407529,747.628862,177.308773,4.495709e+06,0.578141,3.711159,1.627503
min,2.740000e+02,1.000000,0.000000,0.000000,0.000000,4.142174e+06,-7.000000,3.000000,0.000000
25%,6.532920e+05,246.000000,39.000000,15.000000,0.000000,3.289111e+07,0.000000,13.000000,0.000000
50%,1.996838e+06,1145.000000,145.000000,96.000000,3.000000,3.556317e+07,0.000000,17.000000,0.000000
75%,3.545704e+06,5206.000000,496.000000,375.000000,20.000000,3.699112e+07,0.000000,19.000000,1.000000
max,6.449207e+06,287170.000000,23103.000000,7886.000000,2692.000000,3.776986e+07,3.000000,20.000000,11.000000


In [14]:
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(users_posts))
outliers= users_posts[~(z<3).all(axis=1)]
outliers

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
1,450456,7746,3598,256,50,4142174,0,15,0
2,1517244,4172,224,1362,32,37211628,2,18,0
15,2656101,2905,212,572,7,37252377,2,19,0
23,1847261,3417,175,355,48,37390645,2,14,0
32,1076743,7425,1225,454,121,15036385,0,16,0
...,...,...,...,...,...,...,...,...,...
1006,3374382,373,35,14,0,36503270,0,16,6
1036,875393,417,53,236,3,36718577,0,18,6
1051,1551819,147,9,34,0,35987581,0,17,8
1054,4556923,17,13,0,0,36695604,0,20,9


In [16]:
outliers.to_csv('C:/Users/victo/Desktop/IronHack/13-lab-pandas-data-cleaning/your-code/outliers.csv')